# Generate metrics in Table 7.SM.7

The script that pre-dated this notebook, and the calculations that now exist in `src/ar6/metrics`, were originally provided by Bill Collins (University of Reading, UK). 

I have modified them slightly and added documentation.

Theme Song: Low Tide<br>
Artist: Isis, Aereogramme<br>
Album: In The Fishtank 14<br>
Released: 2006

In [1]:
from fair.forcing.ghg import meinshausen

from tqdm.notebook import tqdm
import pandas as pd
import numpy as np

from ar6.metrics.halogen_generic import halogen_analytical
from ar6.metrics.co2 import co2_analytical
from ar6.metrics.ch4 import ch4_analytical
from ar6.metrics.n2o import n2o_analytical
from ar6.metrics.gasser import carbon_cycle_adjustment

## Grab raw data, and define output tables

In [2]:
# Input data file
# note, this is a file that has been cleaned up and standardised by Bill Collins
# original file is from Hodnebrog et al. 2020, Reviews of Geophysics
# https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2019RG000691
# supplementary table S1
halo_file = "../data_input/Hodnebrog_et_al_2020_revgeo/hodnebrog20.csv"

In [3]:
# extract raw data
halo = np.genfromtxt(
    halo_file,
    delimiter=',',
    comments='£', # seemed to work better than None
    names=True,
    dtype="U30, U20, U20, U30, f8, f8, f8",
    usecols = (0, 1, 2, 3, 4, 5, 6)
)

In [4]:
# Allocate output table
num_halo = np.size(halo)
names = ['Name', 'CASRN', 'Acronym', 'Formula', 'Lifetime',
         'Radiative_efficiency', 'AGWP20', 'GWP20', 'AGWP100', 'GWP100', 'AGWP500', 'GWP500',
         'AGTP50', 'GTP50', 'AGTP100', 'GTP100', 'CGTP50', 'CGTP100']
formats = np.concatenate((np.repeat('U30',4), np.repeat('f8', 14)))
table = np.zeros(
    num_halo+3,
    dtype={
        'names': names,
        'formats': formats
    }
)

## Define baseline GHG concentrations

In [5]:
co2 = 409.9
ch4 = 1866.3
n2o = 332.1

## For non-CO2 gases, the metrics require a carbon cycle adjustment

This has to be calculated using a numerical convolution, and we define a timestep of 0.1 years as well as our longest time horizon of interest

In [6]:
ts_per_year = 10
H_max = 500  # years

## First let's get the metrics for CO2

In [7]:
rf_co2, agwp_co2, agtp_co2, iagtp_co2 = co2_analytical(
    np.linspace(0, H_max, H_max * ts_per_year + 1),
    d=np.array([3.424102092311, 285.003477841911]),
    #q=np.array([0.443767728883447, 0.319591049742508]),    # this is what is in Zeb's file sent 25 Feb
    q = np.array([0.443767728883447, 0.313998206372015]),   # this is what Bill used
    co2=co2, n2o=n2o,
    a=np.array([0.2173, 0.2240, 0.2824, 0.2763]),
    alpha_co2=np.array([0, 394.4, 36.54, 4.304]),
    co2_ra = 0.05
)

In [8]:
# Save to first row of output
table[0]['Name'] = 'Carbon dioxide'
table[0]['Formula'] = 'CO2'
table[0]['Lifetime'] = np.nan
table[0]['Radiative_efficiency'] = meinshausen(np.array([co2+0.001, 1866.3, n2o]), np.array([co2, 1866.3, n2o]), scale_F2x=False)[0] * 1.05
table[0]['AGWP20'] = agwp_co2[20 * ts_per_year]
table[0]['GWP20'] = 1.
table[0]['AGWP100'] = agwp_co2[100 * ts_per_year]
table[0]['GWP100'] = 1.
table[0]['AGWP500'] = agwp_co2[500 * ts_per_year]
table[0]['GWP500'] = 1.
table[0]['AGTP50'] = agtp_co2[50 * ts_per_year]
table[0]['GTP50'] = 1.
table[0]['AGTP100'] = agtp_co2[100 * ts_per_year]
table[0]['GTP100'] = 1.
table[0]

('Carbon dioxide', '', '', 'CO2', nan, 1.33449857e-05, 2.43362466e-14, 1., 8.94651231e-14, 1., 3.13800615e-13, 1., 4.27703608e-16, 1., 3.94597382e-16, 1., 0., 0.)

## Then CH4

In [9]:
rf_ch4, agwp_ch4, agtp_ch4, iagtp_ch4 = ch4_analytical(
    np.linspace(0, H_max, H_max * ts_per_year + 1),
    d=np.array([3.424102092311, 285.003477841911]),
    #q=np.array([0.443767728883447, 0.319591049742508]),    # this is what is in Zeb's file sent 25 Feb
    q = np.array([0.443767728883447, 0.313998206372015]),   # this is what Bill used
    co2=co2, ch4=ch4, n2o=n2o,
    ch4_ra = -0.14,
    alpha_ch4 = 11.8,
    ch4_o3=1.4e-4,
    ch4_h2o=0.00004
)

### For non-CO2 metrics, calculate the carbon cycle adjustment

This is quite slow, because every timestep has to be calculated

In [10]:
rf_cc, agwp_cc, agtp_cc = carbon_cycle_adjustment(
    np.linspace(0, H_max, H_max * ts_per_year + 1),
    agtp_ch4,
    co2=co2,
    n2o=n2o,
    co2_ra = 0.05,
    d = np.array([3.424102092311, 285.003477841911]),
    #q=np.array([0.443767728883447, 0.319591049742508]),    # this is what is in Zeb's file sent 25 Feb
    q = np.array([0.443767728883447, 0.313998206372015]),   # this is what Bill used
)

In [11]:
# Save to second row of output
table[1]['Name'] = 'Methane'
table[1]['Formula'] = 'CH4'
table[1]['Lifetime'] = 11.8
table[1]['Radiative_efficiency'] = meinshausen(np.array([co2, ch4+1, n2o]), np.array([co2, ch4, n2o]), scale_F2x=False)[1] * 0.86
table[1]['AGWP20'] = agwp_ch4[20 * ts_per_year] + agwp_cc[20 * ts_per_year]
table[1]['GWP20'] = (agwp_ch4[20 * ts_per_year]+agwp_cc[20 * ts_per_year])/agwp_co2[20 * ts_per_year]
table[1]['AGWP100'] = agwp_ch4[100 * ts_per_year] + agwp_cc[100 * ts_per_year]
table[1]['GWP100'] = (agwp_ch4[100 * ts_per_year]+agwp_cc[100 * ts_per_year])/agwp_co2[100 * ts_per_year]
table[1]['AGWP500'] = agwp_ch4[500 * ts_per_year] + agwp_cc[500 * ts_per_year]
table[1]['GWP500'] = (agwp_ch4[500 * ts_per_year]+agwp_cc[500 * ts_per_year])/agwp_co2[500 * ts_per_year]
table[1]['AGTP50'] = agtp_ch4[50 * ts_per_year] + agtp_cc[50 * ts_per_year]
table[1]['GTP50'] = (agtp_ch4[50 * ts_per_year]+agtp_cc[50 * ts_per_year])/agtp_co2[50 * ts_per_year]
table[1]['AGTP100'] = agtp_ch4[100 * ts_per_year] + agtp_cc[100 * ts_per_year]
table[1]['GTP100'] = (agtp_ch4[100 * ts_per_year]+agtp_cc[100 * ts_per_year])/agtp_co2[100 * ts_per_year]
### and also CGTP for methane
rf_cc, agwp_cc, agtp_cc = carbon_cycle_adjustment(
    np.linspace(0, H_max, H_max * ts_per_year + 1),
    iagtp_ch4,
    co2=co2,
    n2o=n2o,
    co2_ra = 0.05,
    d = np.array([3.424102092311, 285.003477841911]),
    #q=np.array([0.443767728883447, 0.319591049742508]),    # this is what is in Zeb's file sent 25 Feb
    q = np.array([0.443767728883447, 0.313998206372015]),   # this is what Bill used
)
table[1]['CGTP50'] = (iagtp_ch4[50 * ts_per_year]+agtp_cc[50 * ts_per_year])/agtp_co2[50 * ts_per_year]
table[1]['CGTP100'] = (iagtp_ch4[100 * ts_per_year]+agtp_cc[100 * ts_per_year])/agtp_co2[100 * ts_per_year]
table[1]

('Methane', '', '', 'CH4', 11.8, 0.00038864, 1.97607794e-12, 81.19896117, 2.49242801e-12, 27.85921399, 2.49553803e-12, 7.95262314, 4.72554957e-15, 11.04865493, 2.12177589e-15, 5.3770653, 2732.01949364, 3323.33972966)

## Then, N2O

In [12]:
rf_n2o, agwp_n2o, agtp_n2o, iagtp_n2o = n2o_analytical(
    np.linspace(0, H_max, H_max * ts_per_year + 1),
    d=np.array([3.424102092311, 285.003477841911]),
    #q=np.array([0.443767728883447, 0.319591049742508]),    # this is what is in Zeb's file sent 25 Feb
    q = np.array([0.443767728883447, 0.313998206372015]),   # this is what Bill used
    co2=co2, ch4=ch4, n2o=n2o
)
rf_cc, agwp_cc, agtp_cc = carbon_cycle_adjustment(
    np.linspace(0, H_max, H_max * ts_per_year + 1),
    agtp_n2o,
    co2=co2,
    n2o=n2o,
    co2_ra = 0.05,
    d = np.array([3.424102092311, 285.003477841911]),
    #q=np.array([0.443767728883447, 0.319591049742508]),    # this is what is in Zeb's file sent 25 Feb
    q = np.array([0.443767728883447, 0.313998206372015]),   # this is what Bill used
)

In [13]:
rf_n2o, agwp_n2o, agtp_n2o, iagtp_n2o

(array([3.56285141e-13, 3.55958424e-13, 3.55632006e-13, ...,
        3.63427728e-15, 3.63094461e-15, 3.62761499e-15]),
 array([0.00000000e+00, 3.56121758e-14, 7.11916948e-14, ...,
        3.84389442e-11, 3.84393074e-11, 3.84396703e-11]),
 array([0.00000000e+00, 4.58785456e-15, 9.04056787e-15, ...,
        1.29540136e-14, 1.29489266e-14, 1.29438418e-14]),
 array([0.00000000e+00, 2.30535424e-16, 9.13066332e-16, ...,
        2.59046361e-11, 2.59059313e-11, 2.59072259e-11]))

In [14]:
# Save to third row of output
table[2]['Name'] = 'Nitrous oxide'
table[2]['Formula'] = 'N2O'
table[2]['Lifetime'] = 109
table[2]['Radiative_efficiency'] = meinshausen(np.array([co2, ch4, n2o+1]), np.array([co2, ch4, n2o]), scale_F2x=False)[2] * 1.07
table[2]['AGWP20'] = agwp_n2o[20 * ts_per_year] + agwp_cc[20 * ts_per_year]
table[2]['GWP20'] = (agwp_n2o[20 * ts_per_year]+agwp_cc[20 * ts_per_year])/agwp_co2[20 * ts_per_year]
table[2]['AGWP100'] = agwp_n2o[100 * ts_per_year] + agwp_cc[100 * ts_per_year]
table[2]['GWP100'] = (agwp_n2o[100 * ts_per_year]+agwp_cc[100 * ts_per_year])/agwp_co2[100 * ts_per_year]
table[2]['AGWP500'] = agwp_n2o[500 * ts_per_year] + agwp_cc[500 * ts_per_year]
table[2]['GWP500'] = (agwp_n2o[500 * ts_per_year]+agwp_cc[500 * ts_per_year])/agwp_co2[500 * ts_per_year]
table[2]['AGTP50'] = agtp_n2o[50 * ts_per_year] + agtp_cc[50 * ts_per_year]
table[2]['GTP50'] = (agtp_n2o[50 * ts_per_year]+agtp_cc[50 * ts_per_year])/agtp_co2[50 * ts_per_year]
table[2]['AGTP100'] = agtp_n2o[100 * ts_per_year] + agtp_cc[100 * ts_per_year]
table[2]['GTP100'] = (agtp_n2o[100 * ts_per_year]+agtp_cc[100 * ts_per_year])/agtp_co2[100 * ts_per_year]
table[2]

('Nitrous oxide', '', '', 'N2O', 109., 0.00319551, 6.65001314e-12, 273.25549608, 2.44553474e-11, 273.35062615, 4.07045635e-11, 129.71473478, 1.24163669e-13, 290.30306731, 9.19268626e-14, 232.96369098, 0., 0.)

## Now the halogens - this will take a while

### CFC11 and CFC12 need a 12% rapid adjustment

See section 7.3. Although CFC11 is assessed to be 13% for ERF, we use 12% here.

In [15]:
i_out=3
for ispec in tqdm(np.arange(0, num_halo)):
    name = halo[ispec]['Name']
    alpha = halo[ispec]['Lifetime_yr']
    re = halo[ispec]['RE_W_m2_ppb1']
    if (halo[ispec]['Acronym']=='CFC-11' or halo[ispec]['Acronym'] == 'CFC-12'):
        cfc_ra = 0.12
    else:
        cfc_ra = 0
    mass = halo[ispec]['Molar_mass']
    print(name)
    table[i_out]['Name'] = name
    table[i_out]['CASRN'] = halo[ispec]['CASRN']
    table[i_out]['Acronym'] = halo[ispec]['Acronym']
    table[i_out]['Formula'] = halo[ispec]['Formula']
    table[i_out]['Lifetime'] = alpha
    table[i_out]['Radiative_efficiency'] = re * (1 + cfc_ra)
    if not np.isnan(alpha):
#  Pulse
        rf, agwp, agtp, iagtp = halogen_analytical(
            np.linspace(0, H_max, H_max * ts_per_year + 1),
            alpha,
            re,
            mass,
            d=np.array([3.424102092311, 285.003477841911]),
            #q=np.array([0.443767728883447, 0.319591049742508]),    # this is what is in Zeb's file sent 25 Feb
            q = np.array([0.443767728883447, 0.313998206372015]),   # this is what Bill used
            halogen_ra = cfc_ra
        )
        rf_cc, agwp_cc, agtp_cc = carbon_cycle_adjustment(
            np.linspace(0, H_max, H_max * ts_per_year + 1),
            agtp,
            co2=co2,
            n2o=n2o,
            co2_ra = 0.05,
        d = np.array([3.424102092311, 285.003477841911]),
        #q=np.array([0.443767728883447, 0.319591049742508]),    # this is what is in Zeb's file sent 25 Feb
        q = np.array([0.443767728883447, 0.313998206372015]),   # this is what Bill used
        )
        table[i_out]['AGWP20'] = (agwp[20 * ts_per_year]+agwp_cc[20 * ts_per_year])
        table[i_out]['GWP20'] = (agwp[20 * ts_per_year]+agwp_cc[20 * ts_per_year])/agwp_co2[20 * ts_per_year]
        table[i_out]['AGWP100'] = (agwp[100 * ts_per_year]+agwp_cc[100 * ts_per_year])
        table[i_out]['GWP100'] = (agwp[100 * ts_per_year]+agwp_cc[100 * ts_per_year])/agwp_co2[100 * ts_per_year]
        table[i_out]['AGWP500'] = (agwp[500 * ts_per_year]+agwp_cc[500 * ts_per_year])
        table[i_out]['GWP500'] = (agwp[500 * ts_per_year]+agwp_cc[500 * ts_per_year])/agwp_co2[500 * ts_per_year]
        table[i_out]['AGTP50'] = (agtp[50 * ts_per_year]+agtp_cc[50 * ts_per_year])
        table[i_out]['GTP50'] = (agtp[50 * ts_per_year]+agtp_cc[50 * ts_per_year])/agtp_co2[50 * ts_per_year]
        table[i_out]['AGTP100'] = (agtp[100 * ts_per_year]+agtp_cc[100 * ts_per_year])
        table[i_out]['GTP100'] = (agtp[100 * ts_per_year]+agtp_cc[100 * ts_per_year])/agtp_co2[100 * ts_per_year]
# Step
        if alpha < 20.:
            rf_cc, agwp_cc, agtp_cc = carbon_cycle_adjustment(
                np.linspace(0, H_max, H_max * ts_per_year + 1),
                iagtp,
                co2=co2,
                n2o=n2o,
                co2_ra = 0.05,
                d = np.array([3.424102092311, 285.003477841911]),
                #q=np.array([0.443767728883447, 0.319591049742508]),    # this is what is in Zeb's file sent 25 Feb
                q = np.array([0.443767728883447, 0.313998206372015]),   # this is what Bill used
            )
            table[i_out]['CGTP50'] = (iagtp[50 * ts_per_year]+agtp_cc[50 * ts_per_year])/agtp_co2[50 * ts_per_year]
            table[i_out]['CGTP100'] = (iagtp[100 * ts_per_year]+agtp_cc[100 * ts_per_year])/agtp_co2[100 * ts_per_year]
        else:
            table[i_out]['CGTP50'] = np.nan 
            table[i_out]['CGTP100'] = np.nan
        i_out += 1

  0%|          | 0/642 [00:00<?, ?it/s]

Chlorofluorocarbons
Trichlorofluoromethane
Dichlorodifluoromethane
Chlorotrifluoromethane
1:2-Difluoro-1:1:2:2-tetrachlo
2:2-Difluoro-1:2:2:2-tetrachlo
1:1:2-Trichloro-1:2:2-trifluor
1:1:1-Trichloro-2:2:2-trifluor
1:2-Dichloro-1:1:2:2-tetrafluo
1:1-Dichloro-1:2:2:2-tetrafluo
1-Chloro-1:1:2:2:2-pentafluoro
(E)-1:2-Dichlorohexafluorocycl
(Z)-1:2-Dichlorohexafluorocycl
1:2-dichloro-1:2-difluoroethen
1:1-dichloro-2:2-difluoroethen
1:1:2-trichloro-2-fluoroethene
Chlorotrifluoroethylene

Hydrochlorofluorocarbons
Dichlorofluoromethane
Chlorodifluoromethane
Chlorofluoromethane
1:1:2:2-Tetrachloro-1-fluoroet
1:1:2-Trichloro-2:2-difluoroet
1:1:2-Trichloro-1:2-difluoroet
2:2-Dichloro-1:1:1-trifluoroet
1:2-Dichloro-1:1:2-trifluoroet
2-Chloro-1:1:1:2-tetrafluoroet
1-Chloro-1:1:2:2-tetrafluoroet
1:2-Dichloro-1:2-difluoroethan
1:1-Dichloro-2:2-difluoroethan
1:1-Dichloro-1:2-difluoroethan
2-Chloro-1:1:1-trifluoroethane
1:2-Dichloro-1-difluoroethane
1:1-Dichloro-1-fluoroethane
1-Chloro-1:1-difluoroetha

Ethyl methyl ether
Methyl butyl ether
3-Pentanol
5-Methyl-2-hexanone
Methyl methacrylate
Ethyl Prop-2-enoate
Methyl propanoate
Acetic acid ethyl ether
2-Methyl propanoic acid
Methyl salicylate
Methoxyethene
Propanal
3-Methoxyphenol
2-Methoxyphenol
3-Methylfuran
5-Nonanol
2-Propenoic acid
2-Oxopropanal
1:3:3-trimethyl-2-oxabicyclo[2
Ethyl benzoate
Furfural
(2E)-3:7-dimethyl-2:6-octadien
2-Hydroxyacetaldehyde
Ethandial
Hexyl acetate
Octanoic acid
Isopentyl acetate
Methyl-3-oxobutanoate
Phenol
4_methyl-1:3-dioxolan-2-one
Pentenoic acid
2-Propenol
Acetic anhydride
3-Methyl-2-pentanone
(3-hydroxy-2:2:4-trimethylpent
2-Methyl-2-propenal
2-Methyl-1-propanal
2-Methoxyethanol
2-Hexanone
2-Ethyl-1-hexanol
2-Ethoxyethyl acetate
2-Butoxyethanol
1:4-Dioxane
2-Methyloxirane
2-Ethyloxirane
1-Propanol
1-Hexanol
1-Hexanoic acid
1-Heptanol
Butyl acetate
Butanal
Butanoic acid
Diethylketone
Diisopropyl ether
Dimethoxymethane
Dimethyl carbonate
1-(3-methoxypropoxy)propan-1-o
Dipropyl ether
Ethyl butyrate
E

## Save the table, overwriting Bill's original

- Formal comparison to be done offline
- Remembering that CGTP values and metrics for CFC-11 and CFC-12 are both wrong in the March 2021 version
- One other thing to watch out for is that the radiative efficiencies of CO2, CH4 and N2O are too small to be expressed as 3 dp sensibly and will need to be hand-edited later in the final table.

In [23]:
fmt = '%30s'+', %20s'*2+', %30s'+', %5.3f'*2+ ', %7.5e, %5.3f'*5+', %5.3f, %5.3f'

In [24]:
np.savetxt(
    "../data_input/ghg_properties/metrics_supplement.csv",
    table[:i_out],
    delimiter=',',
    fmt=fmt,
    header=','.join(table.dtype.names)
)